### import library

In [ ]:
%pip install opencv-python
%pip install ultralytics
%pip install scikit-learn

In [6]:
import numpy
numpy.__version__ # check the version 1.23.3 

'1.23.3'

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
import cv2
import shutil
import torch
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

### gpu 포트 바꾸기

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('device :', device)
print('Current :', torch.cuda.current_device())
print('Count :', torch.cuda.device_count())

device : cuda
Current : 0
Count : 1


### 데이터 train, valid, test 나누기, yolo 버전으로 텍스트 데이터 처리(최종 : dacon2)

In [4]:
SEED = 10
BATCH_SIZE = 32
# MODEL = "v2"

In [6]:
if not os.path.exists("/home/jrkim/songjunho/dacon2"):
    os.makedirs("/home/jrkim/songjunho/dacon2")

In [7]:
if not os.path.exists("/home/jrkim/songjunho/dacon2/train"):
    os.makedirs("/home/jrkim/songjunho/dacon2/train")
    
if not os.path.exists("/home/jrkim/songjunho/dacon2/valid"):
    os.makedirs("/home/jrkim/songjunho/dacon2/valid")
    
if not os.path.exists("/home/jrkim/songjunho/dacon2/test"):
    os.makedirs("/home/jrkim/songjunho/dacon2/test")    
    
if not os.path.exists("/home/jrkim/songjunho/dacon2/result"):
    os.makedirs("/home/jrkim/songjunho/dacon2/result")

In [8]:
def make_yolo_dataset(image_paths, txt_paths, type="train"):
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)        
        image_height, image_width, _ = source_image.shape
        
        target_image_path = f"/home/jrkim/songjunho/dacon2/{type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)
        
        if type == "test":
            continue
        
        with open(txt_path, "r") as reader:
            yolo_labels = []
            for line in reader.readlines():
                line = list(map(float, line.strip().split(" ")))
                class_name = int(line[0])
                x_min, y_min = float(min(line[1], line[3],line[5],line[7])), float(min(line[2], line[4], line[6], line[8]))
                x_max, y_max = float(max(line[1], line[3],line[5],line[7])), float(max(line[2], line[4], line[6], line[8]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")
            
        target_label_txt = f"/home/jrkim/songjunho/dacon2/{type}/{os.path.basename(txt_path)}"      
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

In [9]:
image_paths = sorted(glob("/home/jrkim/songjunho/train/*.png"))
txt_paths = sorted(glob("/home/jrkim/songjunho/train/*.txt"))

In [10]:
print(len(image_paths))
print(len(txt_paths))

6481
6481


In [11]:
train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.2, random_state=SEED)
train_images_paths = sorted(train_images_paths)
valid_images_paths = sorted(valid_images_paths)
train_txt_paths = sorted(train_txt_paths)
valid_txt_paths = sorted(valid_txt_paths)

In [12]:
make_yolo_dataset(train_images_paths, train_txt_paths, "train")

100%|██████████| 5184/5184 [34:45<00:00,  2.49it/s]  


In [13]:
make_yolo_dataset(valid_images_paths, valid_txt_paths, "valid")

100%|██████████| 1297/1297 [09:00<00:00,  2.40it/s]


In [14]:
make_yolo_dataset(sorted(glob("/home/jrkim/songjunho/test/*.png")), None, "test")

100%|██████████| 3400/3400 [18:12<00:00,  3.11it/s]


### yaml 파일 생성

In [24]:
with open("/home/jrkim/songjunho/classes.txt", "r") as reader:
    lines = reader.readlines()
    classes = [line.strip().split(",")[1] for line in lines]

In [25]:
yaml_data = {
              "names": classes,
              "nc": len(classes),
              "path": "/home/jrkim/songjunho/dacon2/",
              "train": "train",
              "val": "valid",
              "test": "test"
            }


In [26]:
with open("/home/jrkim/songjunho/dacon2/car_data.yaml", "w") as writer:
    yaml.dump(yaml_data, writer)

### model train

In [3]:
#model = YOLO(f"{MODEL}/train/weights/last.pt")
model = YOLO("yolov8x")
# results = model.train(data="/home/mskang/server/dacon1/car_data.yaml", imgsz=512, epochs=10, batch=16, optimizer="Adam") 25점 train19
# results = model.train(data="/home/jrkim/songjunho/dacon/car_data.yaml", imgsz=640, epochs=100, batch=32, optimizer="Adam") 37점
# results = model.train(data="/home/jrkim/songjunho/dacon2/car_data.yaml", imgsz=640, epochs=200, batch=32, optimizer="Adam", patience = 5, workers = 16, lr0 = 1e-3) epochs = 100 일 때 66점이 최고
results = model.train(data="/home/jrkim/songjunho/dacon2/car_data.yaml", imgsz=640, epochs=200, batch=32, optimizer="Adam", patience = 5, workers = 16, lr0 = 1e-3, pretrained = False, resume = True, augment = True, val =True, cache = True) 

New https://pypi.org/project/ultralytics/8.0.112 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.110 🚀 Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/home/jrkim/songjunho/dacon2/car_data.yaml, epochs=200, patience=5, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=16, project=None, name=None, exist_ok=False, pretrained=False, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agn

In [194]:
def get_test_image_paths(test_image_paths):    
    for i in range(0, len(test_image_paths), 34): ## 34로 해 3400개니까
        yield test_image_paths[i:i+34]

In [5]:
test_image_paths = glob("/home/jrkim/songjunho/dacon2/test/*.png")

In [178]:
model = YOLO('/home/jrkim/songjunho/runs/detect/train27/weights/best.pt')

In [195]:
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/34)):
    model.predict(image, imgsz=640, iou = 0.2, conf = 0.7, save_conf=True, save=False, save_txt=True, exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

186it [00:30,  6.97it/s]Results saved to runs/detect/predict
3386 labels saved to runs/detect/predict/labels
187it [00:30,  7.20it/s]Results saved to runs/detect/predict
3386 labels saved to runs/detect/predict/labels
188it [00:30,  7.49it/s]Results saved to runs/detect/predict
3386 labels saved to runs/detect/predict/labels
189it [00:30,  7.36it/s]Results saved to runs/detect/predict
3386 labels saved to runs/detect/predict/labels
190it [00:30,  6.13it/s]


KeyboardInterrupt: 

In [186]:
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/34)):
    model.predict(image, imgsz=640, iou = 0.2, conf = 0.6, save_conf=True, save=False, save_txt=True, exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

# 200epochs iou = 0.7, conf = 0.6
# iou = 0.7 58점
# iou = 0.7, conf = 0.5 60점
# iou = 0.7 62점
# iou = 0.2 62점

 96%|█████████▌| 96/100 [06:54<00:17,  4.38s/it]Results saved to runs/detect/predict
3384 labels saved to runs/detect/predict/labels
 97%|█████████▋| 97/100 [06:59<00:12,  4.31s/it]Results saved to runs/detect/predict
3385 labels saved to runs/detect/predict/labels
 98%|█████████▊| 98/100 [07:03<00:08,  4.36s/it]Results saved to runs/detect/predict
3385 labels saved to runs/detect/predict/labels
 99%|█████████▉| 99/100 [07:07<00:04,  4.30s/it]Results saved to runs/detect/predict
3386 labels saved to runs/detect/predict/labels
100%|██████████| 100/100 [07:11<00:00,  4.32s/it]


In [187]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):    
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]
    
    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)
    
    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [188]:
infer_txts = glob(f"/home/jrkim/songjunho/runs/detect/predict/labels/*.txt")

In [191]:
len(glob(f"/home/jrkim/songjunho/runs/detect/predict/labels/*.txt"))

3386

In [159]:
# re_lines = []
# for i in range(len(lines)):
#     A = lines[i].split(' ')
#     for j in range(len(lines)):
#         B = lines[j].split(' ')
#         x1 = float(A[1]) - float(B[1])
#         y1 = float(A[2]) - float(B[2]) 
#         if np.sqrt((x1*1920)**2 + (y1*1080)**2) < 150:
#             if A[-1] < B[-1]:
#                 break
#             else: continue
#         else: continue
#     if A[-1] < B[-1]:
#         continue
    
#     re_lines.append(lines[i])


In [189]:
results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    image_height, image_width = cv2.imread(f"/home/jrkim/songjunho/dacon2/test/{base_file_name}.png").shape[:2]        
    with open(infer_txt, "r") as reader:        
        lines = reader.readlines()
        re_lines = []
        for i in range(len(lines)):
            A = lines[i].split(' ')
            for j in range(len(lines)):
                B = lines[j].split(' ')
                x1 = float(A[1]) - float(B[1])
                y1 = float(A[2]) - float(B[2]) 
                if np.sqrt((x1*image_width)**2 + (y1*image_height)**2) < 150:
                    if A[-1] < B[-1]:
                        break
                    else: continue
                else: continue
            if A[-1] < B[-1]:
                continue
            
            re_lines.append(lines[i])
                
        for line in re_lines:
            results.append(yolo_to_labelme(line, image_width, image_height, infer_txt))

100%|██████████| 3386/3386 [03:42<00:00, 15.21it/s]


In [190]:
df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"/home/jrkim/songjunho/dacon2/result/result0603_3.csv", index=False, encoding='utf-8')